In [ ]:
# Installing required libraries
!pip install xlsxwriter
!pip install requests

In [ ]:
#Importing required libraries

from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import datetime
import random
import time
import gspread

In [ ]:
from oauth2client.service_account import ServiceAccountCredentials

scopes :["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/sprea...,"https://www.googleapis.com/auth/drive...","https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json",scope)
client = gspread.authorize(creds)


In [ ]:
sheet = client.open("Master-Excel-sheet-TOI")
target_sheet = sheet.get_worsheet(0)
target_sheet.update.acell('A1' , 'Check')

In [ ]:
#Function to send request with retry logic

def send_request_with_retry(url):
    max_retries = 500
    retries = 0
    while retries < max_retries:
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    return response
            except requests.RequestException as e:
                print(f"Request Failed : {e}")
            retries += 1
            #Generation of a random delay between retries
            delay = 1000 + random.randint(1,1000)
            print(f"Retrying in {delay/1000} seconds....")
            time.sleep(delay/1000)
    #Return if all entries fail
    return None


#Forming a dataframe to store and represent Data
my_columns = ['Date' , 'Heading' , 'Text']
df = pd.Dataframe(columns=my_columns)

#Day loop
New_cell = 'A1'
x = 44561
index = 1
Batch = 0
run = 0
t = x+Batch
limit = 44926

while t <= limit:
    daily_url = f'https://timesofindia.indiatimes.com/2022/1/1/archivelist/year-2022,month-1,starttime-{t}.cms'
    Batch += 1
    t += 1

    # Retrieving links from the daily_url
    response = requests.get(daily_url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    link = soup.find('tr' , class_ = "rightColWrap"
    links = link.find_all('a')

    #Retrieving Data from links
    local_url = ""
    for url in links:
        index += 1
        local_url = url.get('href')
        response_link = requests.get(local_url)
        html_content = response_link.text
        soup_link = BeautifulSoup(html_content, 'html.parser')

         try:
             test = 1
             #Extracting Heading
             headings = []
             for tag in soup_link.find_all(['h1]):
                headings.append(tag.text.strip())
                sorted_headings = sorted(headings)
            for h1 in sorted_headings:
                Heading = h1

            #Extracting Date
            datex = soup_link,find('div' , class_ ='VEOUU')
            i = 18
            date = ""
            while (i < len(datex)):
                date = date + datex[i]
                 i += 1
            #Converting Date Format from January 1, 2021 to 01-01-2021
             date_str = date
             date_obj = datetime.strptime(date_str, "